In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2
import anndata

/Users/yiqiangu/miniconda3/envs/SpaGCN/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [43]:
df = pd.read_csv('/Users/yiqiangu/Desktop/Projects/Robert Modlin/CosMx/1_Raw data/Flat_files_standard_pipeline/Run5342_Group3/Run5342_Group3_exprMat_file.csv')


In [44]:
df1 = df[df['fov'] == 7]

In [31]:
df1

,fov,cell_ID,AATK,ABL1,ABL2,ACE,ACE2,ACKR1,ACKR3,ACKR4,...,NegPrb13,NegPrb14,NegPrb15,NegPrb16,NegPrb18,NegPrb19,NegPrb20,NegPrb21,NegPrb22,NegPrb23
11040,7,0,25,30,33,11,57,21,13,26,...,27,27,14,12,19,16,17,22,17,24
11041,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11042,7,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11043,7,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11044,7,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14762,7,3722,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14763,7,3723,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14764,7,3724,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14765,7,3725,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df1.index = df1.cell_ID

In [46]:
df1.drop(columns=['cell_ID','fov','NegPrb3','NegPrb5','NegPrb6','NegPrb7',
                 'NegPrb8','NegPrb9','NegPrb10','NegPrb11','NegPrb12','NegPrb13',
                 'NegPrb14','NegPrb15','NegPrb16','NegPrb18','NegPrb19','NegPrb20',
                 'NegPrb21','NegPrb22','NegPrb23'], inplace =  True)

In [5]:
# input spatial information of cells
spatial1 = pd.read_csv('/Users/yiqiangu/Desktop/Projects/Robert Modlin/CosMx/1_Raw data/Flat_files_standard_pipeline/Run5342_Group3/Run5342_Group3_metadata_file.csv')

In [24]:
spatial1

,fov,cell_ID,Area,AspectRatio,CenterX_local_px,CenterY_local_px,CenterX_global_px,CenterY_global_px,Width,Height,Mean.MembraneStain,Max.MembraneStain,Mean.PanCK,Max.PanCK,Mean.CD45,Max.CD45,Mean.CD3,Max.CD3,Mean.DAPI,Max.DAPI
cell_ID,,,,,,,,,,,,,,,,,,,,
1,7,1,3051,1.06,1406,3617,-17488.444444,-133016.333333,66,62,3283,12402,16,174,164,1431,127,798,2719,8499
2,7,2,2183,1.43,3330,3627,-15564.444444,-133006.333333,60,42,1339,4127,255,453,81,754,56,490,2142,5860
3,7,3,2624,1.53,3489,3625,-15405.444444,-133008.333333,72,47,2093,6989,2739,7859,448,2138,96,571,1601,5436
4,7,4,1471,0.85,3546,3622,-15348.444444,-133011.333333,45,53,2910,7676,1784,5432,219,1460,107,409,2915,6147
5,7,5,2171,1.53,3606,3625,-15288.444444,-133008.333333,72,47,3081,8762,1661,4583,294,1097,95,526,2731,6295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,7,3722,2420,2.71,71,19,-18823.444444,-136614.333333,95,35,2184,5255,1817,4135,357,929,35,278,1935,3457
3723,7,3723,1103,1.56,453,17,-18441.444444,-136616.333333,50,32,1877,4312,97,1613,89,787,46,325,1343,3217
3724,7,3724,724,1.17,2826,16,-16068.444444,-136617.333333,34,29,1585,2221,350,493,105,350,142,511,1534,4255


In [23]:
spatial1 = spatial1[spatial1['fov'] == 7]

In [21]:
spatial1.index =  spatial1.cell_ID

In [37]:
intersect = spatial1.index.intersection(df1.index)

In [47]:
df1 = df1.loc[intersect]

In [48]:
spatial1 = spatial1.loc[intersect]

In [60]:
# convert df1 to 
ad1 = anndata.AnnData(df1)

In [84]:
# merge spatial information into expression
ad1.obs["x_local_px"]=list(spatial1.loc[:,'CenterX_local_px'])
ad1.obs["y_local_px"]=list(spatial1.loc[:,'CenterY_local_px'])
ad1.obs["x_global_px"]=list(spatial1.loc[:,'CenterX_global_px'])
ad1.obs["y_global_px"]=list(spatial1.loc[:,'CenterY_global_px'])
ad1.obs["fov"] = list(spatial1.loc[:,'fov'])

In [85]:
ad1.obs

,x_local_px,y_local_px,x_global_px,y_global_px,fov
cell_ID,,,,,
1,1406,3617,-17488.444444,-133016.333333,7
2,3330,3627,-15564.444444,-133006.333333,7
3,3489,3625,-15405.444444,-133008.333333,7
4,3546,3622,-15348.444444,-133011.333333,7
5,3606,3625,-15288.444444,-133008.333333,7
...,...,...,...,...,...
3722,71,19,-18823.444444,-136614.333333,7
3723,453,17,-18441.444444,-136616.333333,7
3724,2826,16,-16068.444444,-136617.333333,7


In [93]:
ad1.write_h5ad('ad.h5ad')